# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [5]:
QWERTY_ADJACENCY = {
    'q': 'wa', 'w': 'qase', 'e': 'wsdr', 'r': 'edft', 't': 'rfgy', 'y': 'tghu', 'u': 'yihj', 
    'i': 'uojk', 'o': 'ipkl', 'p': 'ol', 'a': 'qwsz', 's': 'awedxz', 'd': 'serfcx', 'f': 'drtgv', 
    'g': 'ftyhbv', 'h': 'gyujnb', 'j': 'huikmn', 'k': 'jiolm', 'l': 'kop', 'z': 'asx', 'x': 'zsdc', 
    'c': 'xdfv', 'v': 'cfgb', 'b': 'vghn', 'n': 'bhjm', 'm': 'njk'
}

def keyboard_edits(word):
    """Generate candidate words by substituting letters with adjacent keyboard keys."""
    candidates = set()
    for i, c in enumerate(word):
        if c in QWERTY_ADJACENCY:
            for adj in QWERTY_ADJACENCY[c]:
                candidates.add(word[:i] + adj + word[i+1:])
    return candidates


In [6]:
#############################################
# USAGE
# -----
# cd ./data/traintest
# python download_datafiles.py
#############################################

#taken from https://github.com/nsadawi/Download-Large-File-From-Google-Drive-Using-Python
#taken from this StackOverflow answer: https://stackoverflow.com/a/39225039
import requests
import os

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def create_paths(path_: str):
    if not os.path.exists(path_):
        os.makedirs(path_)
        print(f"{path_} created")
    else:
        print(f"{path_} already exists")


if __name__=="__main__":

    """
    All files from https://drive.google.com/drive/folders/1ejKSkiHNOlupxXVDMg67rPdqwowsTq1i?usp=sharing
    will be downloaded to the current folder directory
    """

    # ./

    # download_file_from_google_drive('1ZlEQKf3HMMk66F7DGFPnh-PA2cbt5K0F', 'test.1blm')
    # download_file_from_google_drive('1wZ6nrIYANNN3ZoHgacIg9P3UmHnBb9Wa', 'test.1blm.noise.prob')
    # download_file_from_google_drive('1epwQQjmOZyZL1ptc9mcIFjnwS0vs7L46', 'test.1blm.noise.random')
    # download_file_from_google_drive('1aT3mUfsNtTl51vc-V7kJZeflxZ4BMicD', 'test.1blm.noise.word')

    download_file_from_google_drive('1QxVnFgp0pgEWmS-113SWEjT8tEhXCVF5', 'test.bea4k')
    download_file_from_google_drive('1pnCU3OUSE0lNN1T6qY4WWhtHZsW3cg1c', 'test.bea4k.noise')

    # download_file_from_google_drive('1eXrAPKzfU7E9EZNKMyyanuxL9NMpkvdv', 'test.bea20k')
    # download_file_from_google_drive('178AWu05IzYFBOFYQ0lhkkBQaIACSJzAC', 'test.bea20k.noise')

    # download_file_from_google_drive('10VtrEThrDIiuFJf0gj4LeGDdP-y-yR--', 'test.bea60k')
    # download_file_from_google_drive('16AMIb6FVltgRR8xv8h7qacDUX8cOQK9d', 'test.bea60k.noise')

    # download_file_from_google_drive('192g_5oJn4dro5QJ88Dd8-lN_xKE_lLf0', 'test.bea322')
    # download_file_from_google_drive('1_hka2FOT4FrMvsV3d4Zfi9W8v3oFBRYc', 'test.bea322.noise')

    # download_file_from_google_drive('1v0tRcNZctvVGqrmjlda_6dH8AfZUzFGO', 'test.bea4660')
    # download_file_from_google_drive('1EmuKeNgBRzc760R0dSSuZ36xdikQRlIS', 'test.bea4660.noise')

    # download_file_from_google_drive('1jHR2f3JwnskDphQcaTXr0hLlp60qJxUl', 'test.jfleg')
    # download_file_from_google_drive('1sccH7dRhyctKAIQXBZEBmUWEiTN_-o6q', 'test.jfleg.noise')

    # download_file_from_google_drive('1aWHIxu_BrZIeGRLhID3J_od6shXz3jUb', 'train.1blm')
    # download_file_from_google_drive('16RYImD2esgGwc1nNt3Yf-WR5TU1yQyik', 'train.1blm.noise.prob')
    # download_file_from_google_drive('11FMI2C-ouwaWesTLjfPCXmqeB6HUQHkK', 'train.1blm.noise.random')
    # download_file_from_google_drive('1eRpWqSb7sIm3kgtkdfVTru9YKHSRRrdq', 'train.1blm.noise.word')
    
    # download_file_from_google_drive('1INTWXWO6i1Swthu5ln7REZjGvPFv2hyQ', 'train.bea40k')
    # download_file_from_google_drive('1KTeL8oZ30fVI_QuCW879T-CgfeewvSk9', 'train.bea40k.noise')

    # download_file_from_google_drive('1s6CQ6NlsstCLbLCSEZyP-uvNwx4UwzZT', 'train.moviereviews')
    # download_file_from_google_drive('1xk3jyTkiVEWDsl-Abhc8XUXiIVp_WXCg', 'valid.moviereviews')


    # # wo_context
    # create_paths("./wo_context")

    # download_file_from_google_drive('1uNHQovF2Z0QPp27i2Sq5abVL_f56iNYK', './wo_context/aspell_big')
    # download_file_from_google_drive('19eSfVnX-sIdUnaazEaUCsHsFt8p5qoXZ', './wo_context/aspell_big.noise')
    
    # download_file_from_google_drive('1XqHN1VnVnVnSR4-wF_iI6VUVPf9S_TP3', './wo_context/aspell_small')
    # download_file_from_google_drive('1I9jhthL6y52h8uRuwcnRjhtRbNw3mX8V', './wo_context/aspell_small.noise')
    
    # download_file_from_google_drive('1ptBfh8UvbAUH7K1TIALDZM8CUL_Yhxty', './wo_context/combined_data')
    # download_file_from_google_drive('1bSye_TITRUdO4CUIt9i1R2PkCdQD346h', './wo_context/combined_data.noise')
    
    # download_file_from_google_drive('1bj9zQqntrVRydBn-YHZ0BcT55Xf37jq-', './wo_context/homophones')
    # download_file_from_google_drive('1rL_OdqQgr-kL6X_N94epxzpnIzj_L6y6', './wo_context/homophones.noise')


In [7]:
from collections import Counter

def generate_ngrams(words, n):
    return [tuple(words[i:i+n]) for i in range(len(words)-n+1)]

def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        words = []
        for line in file:
            parts = line.strip().split('\t')  # Split by tab
            words.extend(parts[1:])  # Ignore the first column (frequency count)
    
    trigrams = generate_ngrams(words, 3)
    fourgrams = generate_ngrams(words, 4)
    
    trigram_counts = Counter(trigrams)
    fourgram_counts = Counter(fourgrams)
    
    print("Top Trigrams:")
    for trigram, count in trigram_counts.most_common(10):
        print(trigram, count)
    
    print("\nTop Fourgrams:")
    for fourgram, count in fourgram_counts.most_common(10):
        print(fourgram, count)
    return Counter(trigrams) ,Counter(trigrams)

# Replace 'your_file.txt' with your actual file path
TRIGRAMS , FOURGRAMS = process_file('/kaggle/input/fivegrams/fivegrams (2).txt')


Top Trigrams:
('one', 'of', 'the') 6223
('a', 'lot', 'of') 6144
('the', 'united', 'states') 5256
('of', 'the', 'the') 4992
('out', 'of', 'the') 3283
('be', 'able', 'to') 3061
('going', 'to', 'be') 2864
('to', 'be', 'a') 2831
('the', 'of', 'the') 2816
('some', 'of', 'the') 2615

Top Fourgrams:
('in', 'the', 'united', 'states') 1299
('of', 'the', 'of', 'the') 1055
('of', 'the', 'to', 'the') 996
('of', 'the', 'in', 'the') 963
('of', 'the', 'and', 'the') 795
('in', 'front', 'of', 'the') 715
('as', 'well', 'as', 'the') 654
('the', 'rest', 'of', 'the') 649
('at', 'the', 'end', 'of') 601
('of', 'the', 'on', 'the') 598


In [20]:
import re
from collections import Counter, defaultdict
import heapq
import nltk
from nltk.corpus import gutenberg
# Load a simple word frequency dataset (could be replaced with big.txt or N-gram data)
def load_words(filename='/kaggle/input/traindata/big.txt'):
    with open(filename, 'r', encoding='utf-8') as file:
        text = file.read().lower()
    return Counter(re.findall(r'\w+', text))

WORDS = load_words()  # Word frequency from corpus

# Load bigram frequencies (simplified here; ideally from N-gram dataset)
def generate_bigrams(filename='/kaggle/input/bigrams/bigrams (2).txt'):
    bigrams = defaultdict(int)
    with open(filename, 'r', encoding='ISO-8859-1') as file:
        for line in file:
            count,  w1, w2, = line.split()
            bigrams[(w1.lower(), w2.lower())] = int(count)
    return bigrams
# def generate_bigrams():
#     words = [w.lower() for w in gutenberg.words('austen-emma.txt') if w.isalpha()]
#     bigrams = defaultdict(int)
#     for i in range(len(words) - 1):
#         bigrams[(words[i], words[i + 1])] += 1
#     return bigrams

BIGRAMS = generate_bigrams()
# FIGRAMS = generate_bigrams(filename='/kaggle/input/fivegrams/fivegrams (2).txt')
# BIGRAMS = load_bigrams()

# Edit distance functions (Norvig's approach with Damerau-Levenshtein improvements)
def edits1(word):
    """Generate all edits that are one edit away from `word`."""
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    """Generate all edits that are two edits away from `word`."""
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

def known(words):
    """Return subset of words that appear in the dictionary."""
    return set(w for w in words if w in WORDS)

# Probability based on word frequency
def P(word):
    """Probability of word based on frequency."""
    N = sum(WORDS.values())
    return WORDS[word] / N if word in WORDS else 1e-10

# Bigram probability
def P_bigram(prev_word, word):
    """Conditional probability P(word | prev_word) using bigram data."""
    pair = (prev_word, word)
    if pair in BIGRAMS and prev_word in WORDS:
        return BIGRAMS[pair] / WORDS[prev_word]
    return 0  # Fallback to unigram if bigram not found
def P_trigram(w1, w2, w3):
    """Conditional probability P(w3 | w1, w2) using trirgram data."""
    tri = (w1, w2, w3)
    bi = (w1, w2)
    if tri in TRIGRAMS and bi in BIGRAMS: 
        return TRIGRAMS[tri] / BIGRAMS[bi]
    return 0

def P_fourgram(w1, w2, w3, w4):
    """Conditional probability P(w4 | w1, w2, w3) using fourgram data."""
    quad = (w1, w2, w3, w4)
    tri = (w1, w2, w3)
    if quad in FOURGRAMS and tri in TRIGRAMS:
        return FOURGRAMS[quad] / TRIGRAMS[tri]
    return 0
# def P_bigram(prev_word, word):
#     """Conditional probability P(word | prev_word) using bigram data."""
#     pair = (prev_word, word)
#     if pair in FIGRAMS and prev_word in WORDS:
#         return FIGRAMS[pair] / WORDS[prev_word]
#     return P(word)  # Fallback to unigram if bigram not found

# Context-sensitive correction
def correction1(word, prev_word=None ,fith_prev_word=None):
    """Correct a word with context from the previous word."""
      # No correction needed if word is known
    
    # Generate candidates
    candidates = known([word]) or known(edits1(word)) or known(keyboard_edits(word)) or known(edits2(word)) or {word}
    if word in WORDS:
        return max(candidates, key=lambda w: 0.3 * P_bigram(prev_word, w) + 0.7 * P(w))
    # Score candidates with bigram probability if context is provided
    if prev_word:
        # return max(candidates, key=lambda w: P_bigram(prev_word, w))
        return max(candidates, key=lambda w: 0.7 * P_bigram(prev_word, w) + 0.3 * P(w))

    return max(candidates, key=P)  # Fallback to unigram if no context
def correction2(word, prev_words=None):
    """Correct a word with context from up to the last two words."""
    candidates = known([word]) or known(edits1(word)) or known(keyboard_edits(word)) or known(edits2(word)) or {word}
    
    if word in WORDS:
        return max(candidates, key=lambda w: 0.3 * P(w) + 0.7 * best_ngram_prob(prev_words, w))
    
    if prev_words:
        return max(candidates, key=lambda w: 0.7 * best_ngram_prob(prev_words, w) + 0.3 * P(w))
    
    return max(candidates, key=P)  # Fallback to unigram if no context
def correction2_without_keyboard(word, prev_words=None):
    """Correct a word with context from up to the last two words."""
    candidates = known([word]) or known(edits1(word)) or  known(edits2(word)) or {word}
    
    if word in WORDS:
        return max(candidates, key=lambda w: 0.3 * P(w) + 0.7 * best_ngram_prob(prev_words, w))
    
    if prev_words:
        return max(candidates, key=lambda w: 0.7 * best_ngram_prob(prev_words, w) + 0.3 * P(w))
    
    return max(candidates, key=P)  # Fallback to unigram if no context
def correction3(word, prev_words=None):
        """Correct a word with context from up to the last two words."""
        candidates = known([word]) or known(edits1(word)) or  known(edits2(word)) or {word}
        
        if word in WORDS:
            return max(candidates, key=lambda w: 0.7 * P(w) + 0.3 * best_ngram_prob(prev_words, w))
        
        if prev_words:
            return max(candidates, key=lambda w: 0.3 * best_ngram_prob(prev_words, w) + 0.7 * P(w))
        return max(candidates, key=P)
def best_ngram_prob(prev_words, word):
    """Choose the highest probability n-gram where the last n words sequence exists in n-grams data."""
    n = len(prev_words)
    if n >= 3 and tuple(prev_words[-3:] + [word]) in FOURGRAMS:
        return P_fourgram(*prev_words[-3:], word)
    if n >= 2 and tuple(prev_words[-2:] + [word]) in TRIGRAMS:
        return P_trigram(*prev_words[-2:], word)
    return P_bigram(prev_words[-1], word) if n >= 1 else 0  




# Correct a full line of text
def correct_line(line):
    """Correct a line of text word-by-word with context."""
    words = re.findall(r'\w+', line.lower())
    corrected = []
    for i, word in enumerate(words):
        prev_word = corrected[-1] if i > 0 else None
        corrected.append(correction1(word, prev_word))
    return ' '.join(corrected)

def correct_line2(line):
    """Correct a line of text word-by-word with context from up to the last three words."""
    words = re.findall(r'\w+', line.lower())
    corrected = []
    
    for i, word in enumerate(words):
        prev_words = corrected[-3:]  # Get up to the last three corrected words
        corrected.append(correction2(word, prev_words))  # Pass multiple previous words
    
    return ' '.join(corrected)
def correct_line_without_keyboard(line):
    """Correct a line of text word-by-word with context from up to the last three words."""
    words = re.findall(r'\w+', line.lower())
    corrected = []
    
    for i, word in enumerate(words):
        prev_words = corrected[-3:]  # Get up to the last three corrected words
        corrected.append(correction2_without_keyboard(word, prev_words))  # Pass multiple previous words
    
    return ' '.join(corrected)
def correct_line3(line):
    """Correct a line of text word-by-word with context from up to the last three words."""
    words = re.findall(r'\w+', line.lower())
    corrected = []
    
    for i, word in enumerate(words):
        prev_words = corrected[-3:]  # Get up to the last three corrected words
        corrected.append(correction3(word, prev_words))  # Pass multiple previous words
    
    return ' '.join(corrected)


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

# Justification of Implementation Choices

## 1. N-gram Dataset Selection
**Decision:** I used bigram frequencies from a coca ngrams text corpus as the primary context model, with a fallback to unigram from big.txt probabilities when bigram data is unavailable. Additionally, `correction2` and `correction3` support higher N-grams (trigrams and fourgrams) when available.

**Justification:**
- **Bigrams** provide a practical level of context by capturing the probability of a word given the previous word (e.g., "doing sport" vs. "dying sport"). This is sufficient to disambiguate many context-sensitive corrections, as seen in the examples "dking sport" and "dking species."
- **Higher N-grams** (trigrams and fourgrams) were included in `correction2` and `correction3` to leverage more context when available, improving accuracy for sequences where longer dependencies matter. However, I prioritized bigrams as the default because they require less data and computational overhead compared to trigrams or fourgrams, making the solution scalable yet still context-aware.
- The **fallback to unigram probabilities** ensures the model remains robust even when specific bigram data is missing from the corpus, preventing zero-probability issues.

## 2. Candidate Generation
**Decision:** I generated correction candidates using three methods:
- Words one edit away (`edits1`).
- Words two edits away (`edits2`).
- Words one keyboard-adjacent edit away (`keyboard_edits`), except in `correction2_without_keyboard` and `correction3`.

**Justification:**
- **Edits1 and Edits2:** These methods, inherited from Norvig’s solution, cover common typos such as deletions, insertions, substitutions, and transpositions. Including `edits2` ensures the model can handle more severe typos (e.g., "dking" → "doing"), increasing the chances of finding the correct word.
- **Keyboard-adjacent edits:** Many real-world typos stem from pressing adjacent keys on a QWERTY keyboard (e.g., 'd' instead of 's'). By incorporating `keyboard_edits` in `correction1` and `correction2`, the model prioritizes corrections plausible for such errors, improving accuracy. I omitted this in `correction2_without_keyboard` and `correction3` to explore the trade-off between complexity and performance, hypothesizing that edit-based candidates might suffice with strong N-gram weighting.
- **Limitation to Two Edits:** I avoided generating candidates beyond two edits to keep the search space manageable, as further edits would exponentially increase computation time with diminishing returns in accuracy for most cases.

## 3. Probability Weighting
**Decision:** I implemented different weighting schemes for known and unknown words across the four correction functions:
- `correction1`: 0.3 * P_bigram + 0.7 * P_unigram (known words), 0.7 * P_bigram + 0.3 * P_unigram (unknown words).
- `correction2`: 0.3 * P_unigram + 0.7 * P_ngram (known words), 0.7 * P_ngram + 0.3 * P_unigram (unknown words).
- `correction2_without_keyboard`: Same as `correction2`.
- `correction3`: 0.7 * P_unigram + 0.3 * P_ngram (known words), 0.3 * P_ngram + 0.7 * P_unigram (unknown words).

**Justification:**
- **Known vs. Unknown Words:** For known words (already in the dictionary), I generally gave higher weight to unigram probability to avoid overcorrecting correct words based solely on context. For unknown words (likely typos), I prioritized N-gram probability to leverage context for selecting the best correction.
- **Weight Variations:**
  - In `correction1`, the weights (0.3/0.7 vs. 0.7/0.3) balance context and frequency, reflecting a conservative approach where unigram dominance preserves known words, while bigram dominance guides typo correction.
  - In `correction2`, I emphasized higher N-grams (0.7) to exploit richer context when available, assuming that trigram/fourgram data could better disambiguate complex cases.
  - `correction3` reverses this (0.7 unigram for known words), testing a hypothesis that over-reliance on N-grams might overfit to context, especially with sparse higher N-gram data.
- **Heuristic Choice:** These weights were chosen heuristically as a starting point, based on the intuition that context matters more for errors and frequency matters more for correct words. They could be fine-tuned with evaluation data for optimal performance.

## 4. Four Correction Algorithms
**Decision:** I implemented four correction functions (`correction1`, `correction2`, `correction2_without_keyboard`, `correction3`) to explore different approaches:
- `correction1`: Uses bigrams with keyboard edits.
- `correction2`: Uses up to fourgrams with keyboard edits.
- `correction2_without_keyboard`: Uses up to fourgrams without keyboard edits.
- `correction3`: Uses up to fourgrams with adjusted weights, no keyboard edits.

**Justification:**
- **Exploration of Trade-offs:** Each function tests a different hypothesis:
  - `correction1` mirrors a baseline improvement over Norvig with bigrams and keyboard awareness.
  - `correction2` extends this to higher N-grams, hypothesizing that more context improves accuracy.
  - `correction2_without_keyboard` isolates the effect of keyboard edits, testing if N-gram strength alone suffices.
  - `correction3` adjusts weights to prioritize unigram probability, exploring robustness against sparse N-gram data.
- **Flexibility:** Providing multiple algorithms allows comparison and adaptation based on specific use cases or evaluation results, fulfilling the task’s expectation to innovate beyond Norvig’s solution.

## 5. Handling Unknown Words
**Decision:** For unknown words, I generate candidates from `edits1`, `keyboard_edits` (where applicable), and `edits2`, then select the highest-scoring option based on weighted probabilities.

**Justification:** This approach ensures the model can propose reasonable corrections even for words not in the dictionary, using both edit distance and context to rank candidates. The inclusion of keyboard edits (in `correction1` and `correction2`) enhances realism, while the fallback to unigram probability ensures a default mechanism when context data is lacking.

## 6. Efficiency Considerations
**Decision:** I used `Counter` for unigram frequencies and `defaultdict` for bigram frequencies, avoiding complex models like neural networks.

**Justification:**
- Hash-based structures provide O(1) lookup times, making the correction process fast even for long texts. This is critical for practical spelling correction, aligning with the task’s efficiency improvement goal.
- Simpler frequency-based models were chosen over neural networks due to their lower computational cost and ease of implementation, sufficient for a context-sensitive proof-of-concept.

## 7. No Beam Search
**Decision:** I opted for a greedy, word-by-word correction approach instead of beam search.

**Justification:** Beam search excels in sequence modeling tasks with long dependencies (e.g., translation), but spelling correction typically relies on local context (e.g., the previous word or two). A greedy approach is faster and sufficient for this task, avoiding unnecessary complexity without significant accuracy gains.

## 8. Context Window
**Decision:** `correction1` uses a one-word context (bigrams), while `correction2`, `correction2_without_keyboard`, and `correction3` use up to three previous words (fourgrams).

**Justification:**
- A one-word context balances simplicity and effectiveness for most cases, as seen in "dking sport" vs. "dking species."
- Extending to three words in the other algorithms tests whether additional context improves accuracy, particularly for ambiguous cases, while keeping the model manageable compared to even larger windows.

## 9. Fallback Mechanism
**Decision:** If N-gram data is unavailable, I fall back to unigram probability.

**Justification:** This prevents the model from failing on unseen word pairs, ensuring robustness and a reasonable correction even with sparse training data.

## 10. Case Insensitivity
**Decision:** All text is processed in lowercase.

**Justification:** Spelling correction focuses on word correctness, not case. Handling case separately would complicate the model unnecessarily, and most frequency datasets are case-insensitive.

.

## Summary of Key Improvements Over Norvig’s Solution
- **Context Sensitivity:** Added N-gram probabilities (bigrams and beyond) to consider surrounding words, unlike Norvig’s unigram-only approach.
- **Keyboard Awareness:** Incorporated QWERTY-adjacent edits to model realistic typos (except in two variants).
- **Multiple Algorithms:** Implemented four correction functions to test different weighting and context strategies.
- **Efficiency:** Maintained fast lookups with dictionary-based structures, enhancing scalability.

These choices collectively aim to create a spelling corrector that is accurate, context-sensitive, and efficient, addressing the task’s requirements while introducing meaningful improvements over the original Norvig solution.

More on probability Weighting: Detailed Analysis

### Overview of the Decision
The probability weighting schemes determine how much influence different probabilistic models (unigram, bigram, or higher N-gram probabilities) have on selecting the best correction candidate for a given word. I implemented distinct weighting strategies for **known words** (words already in the dictionary) and **unknown words** (potential typos not in the dictionary) across four correction functions:
- **`correction1`:** 0.3 * P_bigram + 0.7 * P_unigram (known words), 0.7 * P_bigram + 0.3 * P_unigram (unknown words).
- **`correction2`:** 0.3 * P_unigram + 0.7 * P_ngram (known words), 0.7 * P_ngram + 0.3 * P_unigram (unknown words).
- **`correction2_without_keyboard`:** Same as `correction2`.
- **`correction3`:** 0.7 * P_unigram + 0.3 * P_ngram (known words), 0.3 * P_ngram + 0.7 * P_unigram (unknown words).

These weights combine the **unigram probability (P_unigram)**—the frequency of a word in isolation—and the **N-gram probability (P_bigram or P_ngram)**—the likelihood of a word given its preceding context (one word for bigrams, up to three for higher N-grams)—to score correction candidates. The choice of weights directly impacts how the model prioritizes frequency versus context, affecting prediction accuracy, robustness, and behavior in different scenarios.

---

### How Weighting Affects Predictions

#### General Mechanism
For each word in the input text:
1. If the word is **known**, the model evaluates whether to keep it as-is or replace it with a correction candidate based on the weighted probability score.
2. If the word is **unknown**, the model generates correction candidates (via edits or keyboard-adjacent methods) and selects the one with the highest weighted score.
3. The final prediction is the candidate with the maximum combined probability, where the weights dictate the balance between **context (N-gram)** and **frequency (unigram)**.

The differing weights across the functions create distinct prediction behaviors, which I’ll analyze below.

---

#### `correction1`: Balancing Context and Frequency
- **Known Words:** 0.3 * P_bigram + 0.7 * P_unigram
  - **Effect:** This weighting heavily favors unigram probability (70%), meaning the model prioritizes a word’s overall frequency over its contextual fit. For a known word, the model is conservative—it’s less likely to “correct” it unless the bigram probability strongly suggests a better alternative.
  - **Prediction Impact:** If the input is “doing sport” (both words known), the high unigram weight keeps “doing” unless “dying sport” has a significantly higher bigram probability (e.g., in a corpus about mortality in sports). This reduces over-correction of valid words but may miss context-driven errors.
- **Unknown Words:** 0.7 * P_bigram + 0.3 * P_unigram
  - **Effect:** Here, bigram probability dominates (70%), emphasizing context over frequency. For an unknown word like “dking” in “dking sport,” the model prefers candidates like “doing” if “doing sport” has a high bigram probability, even if “dying” is more frequent overall.
  - **Prediction Impact:** This makes predictions context-sensitive, correctly resolving “dking sport” to “doing sport” in a sports context, but it risks overfitting to rare bigrams if the corpus is sparse.

**Overall Behavior:** `correction1` strikes a balance—preserving known words unless context strongly disagrees, while aggressively using context to fix typos. It’s effective for short-context corrections but may falter with ambiguous or sparse bigram data.

---

#### `correction2` and `correction2_without_keyboard`: Leveraging Richer Context
- **Known Words:** 0.3 * P_unigram + 0.7 * P_ngram
  - **Effect:** Higher N-gram probability (up to fourgrams, 70%) takes precedence, meaning context from up to three previous words heavily influences whether a known word is kept or corrected. Unigram probability (30%) acts as a tiebreaker.
  - **Prediction Impact:** For “doing sport” in a longer sequence like “he is doing sport,” if “is doing sport” has a strong trigram probability, “doing” stays. However, if the N-gram strongly favors “is dying sport” (unlikely but possible in a niche corpus), it might overcorrect “doing” to “dying.” This risks altering correct words when N-gram data is noisy or overfitted.
- **Unknown Words:** 0.7 * P_ngram + 0.3 * P_unigram
  - **Effect:** N-gram probability again dominates (70%), using richer context to rank correction candidates for typos. The low unigram weight (30%) ensures frequency plays a minor role.
  - **Prediction Impact:** For “dking” in “he is dking sport,” the model evaluates candidates like “doing” and “dying” based on “is doing sport” or “is dying sport.” If trigrams/fourgrams strongly favor “doing,” it predicts correctly. This excels in disambiguating complex cases but depends heavily on the quality and coverage of N-gram data.

**Overall Behavior:** `correction2` (and its variant) prioritizes context, making it powerful for sequences with clear N-gram patterns (e.g., idiomatic phrases). However, sparse or unreliable higher N-gram data could lead to incorrect predictions, especially for known words.

---

#### `correction3`: Frequency-First Approach
- **Known Words:** 0.7 * P_unigram + 0.3 * P_ngram
  - **Effect:** Unigram probability dominates (70%), making the model highly conservative for known words. N-gram context (up to fourgrams, 30%) has less sway.
  - **Prediction Impact:** For “doing sport,” “doing” is retained unless “doing sport” has an unusually low N-gram probability compared to “dying sport.” This minimizes over-correction but may fail to catch context-driven errors (e.g., “dying species” miskept as “doing species”).
- **Unknown Words:** 0.3 * P_ngram + 0.7 * P_unigram
  - **Effect:** Unlike the other functions, unigram probability (70%) outweighs N-gram context (30%). For unknown words, frequency drives predictions more than context.
  - **Prediction Impact:** For “dking” in “dking sport,” if “doing” is more frequent than “dying” in the unigram data, it’s chosen even if “dying sport” fits the context better. This can lead to context-insensitive corrections (e.g., “doing” over “dying” in “dking species”), reducing accuracy in ambiguous cases.

**Overall Behavior:** `correction3` favors frequency, making it robust against sparse N-gram data but less adept at leveraging context. It’s less likely to overcorrect known words but may produce generic, context-ignorant fixes for typos.

---

### Comparative Effects on Predictions

#### Accuracy in Context-Sensitive Cases
- **`correction1`:** Excels with bigrams (e.g., “dking sport” → “doing sport”), but limited by one-word context.
- **`correction2`:** Leverages richer context (e.g., “he is dking sport” → “he is doing sport”), improving accuracy when N-gram data is reliable, but risks over-correction.
- **`correction3`:** Prioritizes frequency, potentially missing context cues (e.g., “dking species” → “doing species” instead of “dying species”).

#### Robustness to Sparse Data
- **`correction1`:** Fallback to unigram ensures reasonable predictions when bigram data is missing.
- **`correction2`:** Relies on higher N-grams, so sparse trigram/fourgram data could degrade performance unless unigram fallback compensates.
- **`correction3`:** Most robust due to unigram dominance, but at the cost of context sensitivity.

#### Over-Correction Risk
- **`correction1`:** Moderate—conservative for known words, context-driven for unknowns.
- **`correction2`:** Highest—strong N-gram weighting may alter correct words based on misleading context.
- **`correction3`:** Lowest—unigram focus preserves known words but may miss needed corrections.

---

### Practical Implications
- **Use Case Fit:**
  - **`correction1`:** Best for general-purpose correction with moderate context needs and limited data.
  - **`correction2`:** Ideal for applications with rich, reliable N-gram data (e.g., domain-specific texts) where context is critical.
  - **`correction3`:** Suited for noisy or sparse datasets, prioritizing simplicity and frequency over nuanced context.
- **Tuning Potential:** The heuristic weights (e.g., 0.7/0.3) could be optimized with evaluation data (e.g., precision/recall on a misspelling test set) to better balance context and frequency for specific domains.

---

### Conclusion
The weighting schemes in `correction1`, `correction2`, and `correction3` create a spectrum of prediction behaviors:
- **Context-Driven (`correction2`):** Maximizes accuracy in well-defined contexts but risks overfitting.
- **Balanced (`correction1`):** Offers a practical middle ground, suitable for most cases.
- **Frequency-Driven (`correction3`):** Ensures robustness but sacrifices contextual nuance.

These choices affect predictions by shaping how the model interprets the trade-off between a word’s inherent likelihood (unigram) and its fit within the sentence (N-gram), ultimately determining its effectiveness across diverse inputs and data conditions.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [31]:
clean_lines = load_clean_lines('/kaggle/input/tatoeba-english-extracted/eng_sentences.tsv', max_lines=1000)  # Load first 10 sentences
print(len(clean_lines))

1000


## Dataset Description
The dataset used for this evaluation is sourced from tatoeba english, a tab-separated values (TSV) file containing English sentences extracted from the Tatoeba project. Tatoeba is a collaborative, open-source collection of sentences and translations, widely used for natural language processing tasks. This specific file includes English sentences, likely with columns for sentence IDs, language codes, and the sentence text itself. For the evaluation, I loaded a subset of up to 100,000 sentences (though limited by `max_lines` in the code), removed punctuation, and converted the text to lowercase to focus on word-level spelling correction. I then introduced synthetic noise (e.g., random edits with a probability of 0.2) to generate a test set of noisy sentences, which were used to compare the performance of my four correction algorithms against Norvig’s baseline solution.


## Evaluation Issue
A key limitation of this evaluation is the use of random noise to generate the test set, rather than simulating human-like mistakes. The `add_noise` function applies random edits (e.g., insertions, deletions, substitutions) with a probability of 0.2, which does not fully reflect common human typing errors, such as pressing adjacent keys on a keyboard (e.g., 'd' instead of 's'). This explains why the keyboard-aware method (`correction2`) and the no-keyboard method (`correction2_without_keyboard`) perform identically in terms of accuracy (both at 0.900024). The random noise does not preferentially generate keyboard-adjacent errors, reducing the advantage of the keyboard-aware approach. To better evaluate the keyboard edit feature, a test set with human-like typos (e.g., based on QWERTY adjacency) would be more appropriate.

In [24]:
import random
def add_noise(word, noise_prob=0.2):
    """Add random noise to a word with given probability."""
    if random.random() > noise_prob:
        return word
    edits = list(edits1(word))
    return random.choice(edits) if edits else word

def generate_test_set(clean_lines, noise_prob=0.35):
    """Generate noisy versions of clean lines."""
    noisy_lines = []
    for line in clean_lines:
        noisy_words = [add_noise(word, noise_prob) for word in re.findall(r'\w+', line.lower())]
        noisy_lines.append(' '.join(noisy_words))
    return noisy_lines

# Load clean lines from extracted Tatoeba eng_sentences.tsv
def load_clean_lines(filename='/kaggle/input/tatoeba-english-extracted/eng_sentences.tsv', max_lines=1000):
    """Load a subset of English sentences from the uncompressed TSV file and remove punctuation."""
    clean_lines = []
    with open(filename, 'r', encoding='utf-8') as file:
        for i, line in enumerate(file):
            if i >= max_lines:  # Limit to max_lines for testing
                break
            parts = line.strip().split('\t')
            if len(parts) == 3:  # Ensure valid TSV row
                sentence = parts[2]  # Third column is the text
                # Remove all punctuation using regex
                cleaned_sentence = re.sub(r'[^\w\s]', '', sentence)
                clean_lines.append(cleaned_sentence)
    return clean_lines

# Load the dataset
clean_lines = load_clean_lines('/kaggle/input/tatoeba-english-extracted/eng_sentences.tsv', max_lines=1000)  # Load first 10 sentences
noisy_lines = generate_test_set(clean_lines, noise_prob=0.2)

# Compare with Norvig's solution
def norvig_correct_line(line):
    words = re.findall(r'\w+', line.lower())
    return ' '.join(max(known([w]) or known(edits1(w)) or known(edits2(w)) or {w}, key=P) for w in words)

# Accuracy metric
def accuracy(corrected, target):
    correct_words = sum(1 for c, t in zip(corrected.split(), target.split()) if c.lower() == t.lower())
    return correct_words / len(target.split())

# Run evaluation
print("Evaluation Results:")
total_my_acc_1 = 0
total_my_acc_2 = 0
total_my_acc2_without_keyboard = 0
total_my_acc_3 = 0
total_norvig_acc = 0

for i, (clean, noisy) in enumerate(zip(clean_lines, noisy_lines)):
    my_corrected_1 = correct_line(noisy)
    my_corrected_2 = correct_line2(noisy)
    my_corrected_without_keyboard = correct_line_without_keyboard(noisy)
    my_corrected_3 = correct_line3(noisy)
    norvig_corrected = norvig_correct_line(noisy)
    
    my_acc_1 = accuracy(my_corrected_1, clean)
    my_acc_2 = accuracy(my_corrected_2, clean)
    my_acc2_without_keyboard = accuracy(my_corrected_without_keyboard, clean)
    my_acc_3 = accuracy(my_corrected_3, clean)
    norvig_acc = accuracy(norvig_corrected, clean)
    
    # Accumulate accuracies separately
    total_my_acc_1 += my_acc_1
    total_my_acc_2 += my_acc_2
    total_my_acc2_without_keyboard += my_acc2_without_keyboard
    total_my_acc_3 += my_acc_3
    total_norvig_acc += norvig_acc
    
    if i % 150 == 0:
        print(f"Clean: {clean}")
        print(f"Noisy: {noisy}")
        print(f"My Corrected 1: {my_corrected_1} (Acc: {my_acc_1:.2f})")
        print(f"My Corrected 2: {my_corrected_2} (Acc: {my_acc_2:.2f})")
        print(f"My Corrected (No Keyboard): {my_corrected_without_keyboard} (Acc: {my_acc2_without_keyboard:.2f})")
        print(f"My Corrected 3: {my_corrected_3} (Acc: {my_acc_3:.2f})")
        print(f"Norvig Corrected: {norvig_corrected} (Acc: {norvig_acc:.2f})\n")

# Calculate and print average accuracies
num_lines = len(clean_lines)
print(f"Average My Accuracy 1: {total_my_acc_1 / num_lines:.6f}")
print(f"Average My Accuracy 2: {total_my_acc_2 / num_lines:.6f}")
print(f"Average My Accuracy 2 (No Keyboard): {total_my_acc2_without_keyboard / num_lines:.6f}")
print(f"Average My Accuracy 3: {total_my_acc_3 / num_lines:.6f}")
print(f"Average Norvig Accuracy: {total_norvig_acc / num_lines:.6f}")

Evaluation Results:
Clean: Lets try something
Noisy: lets try something
My Corrected 1: lets try something (Acc: 1.00)
My Corrected 2: lets try something (Acc: 1.00)
My Corrected (No Keyboard): lets try something (Acc: 1.00)
My Corrected 3: lets try something (Acc: 1.00)
Norvig Corrected: lets try something (Acc: 1.00)

Clean: I dont like you anymore
Noisy: x doqnt like you anxymore
My Corrected 1: x dont like you anymore (Acc: 0.80)
My Corrected 2: x dont like you anymore (Acc: 0.80)
My Corrected (No Keyboard): x dont like you anymore (Acc: 0.80)
My Corrected 3: x dont like you anymore (Acc: 0.80)
Norvig Corrected: x dont like you anymore (Acc: 0.80)

Clean: How do you feel he inquired
Noisy: how do you feel ve inquiredw
My Corrected 1: how do you feel ve inquired (Acc: 0.83)
My Corrected 2: how do you feel ve inquired (Acc: 0.83)
My Corrected (No Keyboard): how do you feel ve inquired (Acc: 0.83)
My Corrected 3: how do you feel ve inquired (Acc: 0.83)
Norvig Corrected: how do you fee

## Visualization of First Test Results (Tatoeba with Random Noise)

Below is a bar plot comparing the average accuracies of the spelling correction methods on the Tatoeba dataset with synthetic random noise:

<img src="tatoeba_accuracy_comparison_small.png" alt="Average Accuracy Comparison (Tatoeba)" width="1200">

### Interpretation
- `Correction 1` achieves the highest accuracy (0.910313), outperforming all other methods.
- `Correction 2` and `Correction 2 (No Keyboard)` tie at the lowest accuracy (0.900024), indicating no benefit from keyboard-aware edits with random noise.
- `Correction 3` (0.900676) and Norvig (0.901572) are closely matched, with Norvig slightly ahead of the higher N-gram methods.

In [32]:
def load_bea4k_dataset(noisy_file='/kaggle/input/bea4k/test.bea4k.noise', clean_file='/kaggle/input/bea4k/test.bea4k', max_lines=100):
    noisy_lines = []
    clean_lines = []
    with open(noisy_file, 'r', encoding='utf-8') as nf, open(clean_file, 'r', encoding='utf-8') as cf:
        for i, (n_line, c_line) in enumerate(zip(nf, cf)):
            if i >= max_lines:  # Limit to max_lines
                break
            # Remove punctuation and normalize to lowercase
            clean_text = ' '.join(re.findall(r'\w+', c_line.lower()))
            noisy_text = ' '.join(re.findall(r'\w+', n_line.lower()))
            clean_lines.append(clean_text)
            noisy_lines.append(noisy_text)
    return clean_lines, noisy_lines
clean_lines, noisy_lines = load_bea4k_dataset('/kaggle/working/test.bea4k.noise', '/kaggle/working/test.bea4k', max_lines=100000000)

In [33]:
len(clean_lines),len(noisy_lines)

(4280, 4280)

In [36]:
num_lines = len(clean_lines)

In [37]:
len(clean_lines)

4280

## Second Test Set: BEA-4K Dataset

### Dataset Description
For the second evaluation, I used the BEA-4K dataset, downloaded from Google Drive using the following commands:
- `download_file_from_google_drive('1QxVnFgp0pgEWmS-113SWEjT8tEhXCVF5', 'test.bea4k')` for the clean text file.
- `download_file_from_google_drive('1pnCU3OUSE0lNN1T6qY4WWhtHZsW3cg1c', 'test.bea4k.noise')` for the noisy (error-introduced) text file.

The BEA-4K dataset is a subset of the BEA (Building Educational Applications) dataset, commonly used for grammatical error correction (GEC) and spelling correction research. It contains approximately 4,000 sentence pairs, where `test.bea4k` includes the correct (clean) English sentences, and `test.bea4k.noise` contains the same sentences with human-like errors introduced. These errors are designed to mimic real-world typing mistakes, such as misspellings, keyboard adjacency errors (e.g., "teh" for "the"), and other common typos, rather than purely random edits. The dataset is likely in a plain text format, with one sentence per line, making it straightforward to process for spelling correction tasks.

### Why This Test is Better
The BEA-4K dataset offers several advantages over the previous test set (Tatoeba with synthetic random noise), making it a more robust evaluation for my spelling correction algorithms:
- **Human-Like Errors:** Unlike the random noise approach (e.g., `add_noise` with a 0.2 probability), which applied uniform edit operations (insertions, deletions, substitutions), the BEA-4K noisy data reflects realistic human typing mistakes. This includes keyboard adjacency errors, transposition of letters, and contextually plausible misspellings, aligning better with the intended use case of my keyboard-aware correction methods (`correction1` and `correction2`).
- **Discrimination of Keyboard Methods:** In the first test, `correction2` (with keyboard edits) and `correction2_without_keyboard` performed identically (0.900024), likely because random noise didn’t favor keyboard-specific errors. The BEA-4K dataset’s human-like typos allow me to better evaluate the effectiveness of my keyboard-adjacent edit feature, as it should now outperform the no-keyboard variant in correcting errors like "hte" to "the".
- **Real-World Relevance:** The BEA-4K dataset is curated for evaluating error correction systems, ensuring that the errors are representative of those encountered in real text (e.g., student writing or casual typing). This contrasts with the synthetic noise in the Tatoeba test, which lacked linguistic or typographical realism.
- **Standardized Benchmark:** BEA-4K is a recognized dataset in NLP research, providing a standardized benchmark to compare my algorithms against Norvig’s solution and potentially other state-of-the-art systems. This enhances the credibility and comparability of the results.

By switching to BEA-4K, this second test better assesses the context-sensitivity and keyboard-awareness of my algorithms, addressing the limitations of the first evaluation and offering a more meaningful measure of performance in practical scenarios.

In [38]:
import random
random.seed(42)
import json
def add_noise(word, noise_prob=0.2):
    """Add random noise to a word with given probability."""
    if random.random() > noise_prob:
        return word
    edits = list(edits1(word))
    return random.choice(edits) if edits else word

def generate_test_set(clean_lines, noise_prob=0.35):
    """Generate noisy versions of clean lines."""
    noisy_lines = []
    for line in clean_lines:
        noisy_words = [add_noise(word, noise_prob) for word in re.findall(r'\w+', line.lower())]
        noisy_lines.append(' '.join(noisy_words))
    return noisy_lines

# Load clean lines from extracted Tatoeba eng_sentences.tsv
def load_clean_lines(filename='/kaggle/input/tatoeba-english-extracted/eng_sentences.tsv', max_lines=10):
    """Load a subset of English sentences from the uncompressed TSV file."""
    clean_lines = []
    with open(filename, 'r', encoding='utf-8') as file:
        for i, line in enumerate(file):
            if i >= max_lines:  # Limit to max_lines for testing
                break
            parts = line.strip().split('\t')
            if len(parts) == 3:  # Ensure valid TSV row
                sentence = parts[2]  # Third column is the text
                clean_lines.append(sentence)
    return clean_lines

# Load the dataset
# clean_lines = load_clean_lines('/kaggle/input/tatoeba-english-extracted/eng_sentences.tsv', max_lines=10)  # Load first 10 sentences
# noisy_lines = generate_test_set(clean_lines, noise_prob=0.2)

# Compare with Norvig's solution

def norvig_correct_line(line):
    words = re.findall(r'\w+', line.lower())
    return ' '.join(max(known([w]) or known(edits1(w)) or known(edits2(w)) or {w}, key=P) for w in words)

# Accuracy metric
def accuracy(corrected, target):
    correct_words = sum(1 for c, t in zip(corrected.split(), target.split()) if c.lower() == t.lower())
    return correct_words / len(target.split())

# Run evaluation
print("Evaluation Results:")
total_my_acc_1 = 0
total_my_acc_2 = 0
total_my_acc2_without_keyboard = 0
total_my_acc_3 = 0
total_norvig_acc = 0
for i, (clean, noisy) in enumerate(zip(clean_lines, noisy_lines)):
    my_corrected_1 = correct_line(noisy)
    my_corrected_2 = correct_line2(noisy)
    my_corrected_without_keyboard = correct_line_without_keyboard(noisy)
    my_corrected_3 = correct_line3(noisy)
    norvig_corrected = norvig_correct_line(noisy)
    
    my_acc_1 = accuracy(my_corrected_1, clean)
    my_acc_2 = accuracy(my_corrected_2, clean)
    my_acc2_without_keyboard = accuracy(my_corrected_without_keyboard, clean)
    my_acc_3 = accuracy(my_corrected_3, clean)
    norvig_acc = accuracy(norvig_corrected, clean)
    
    # Accumulate accuracies separately
    total_my_acc_1 += my_acc_1
    total_my_acc_2 += my_acc_2
    total_my_acc2_without_keyboard += my_acc2_without_keyboard
    total_my_acc_3 += my_acc_3
    total_norvig_acc += norvig_acc
    
    if i % 150 == 0:
        print(f"Clean: {clean}")
        print(f"Noisy: {noisy}")
        print(f"My Corrected 1: {my_corrected_1} (Acc: {my_acc_1:.2f})")
        print(f"My Corrected 2: {my_corrected_2} (Acc: {my_acc_2:.2f})")
        print(f"My Corrected (No Keyboard): {my_corrected_without_keyboard} (Acc: {my_acc2_without_keyboard:.2f})")
        print(f"My Corrected 3: {my_corrected_3} (Acc: {my_acc_3:.2f})")
        print(f"Norvig Corrected: {norvig_corrected} (Acc: {norvig_acc:.2f})\n")

# Calculate and print average accuracies
num_lines = len(clean_lines)
print(f"Average My Accuracy 1: {total_my_acc_1 / num_lines:.6f}")
print(f"Average My Accuracy 2: {total_my_acc_2 / num_lines:.6f}")
print(f"Average My Accuracy 2 (No Keyboard): {total_my_acc2_without_keyboard / num_lines:.6f}")
print(f"Average My Accuracy 3: {total_my_acc_3 / num_lines:.6f}")
print(f"Average Norvig Accuracy: {total_norvig_acc / num_lines:.6f}")

Evaluation Results:
Clean: i have just received the letter which lets me know that i have won the first prize
Noisy: i have just recieved the letter which lets me know that i have won the first prize
My Corrected 1: i have just received the letter which lets me know that i have won the first prize (Acc: 1.00)
My Corrected 2: i have just received the letter which lets me know that i have won the first prize (Acc: 1.00)
My Corrected (No Keyboard): i have just received the letter which lets me know that i have won the first prize (Acc: 1.00)
My Corrected 3: i have just received the letter which lets me know that i have won the first prize (Acc: 1.00)
Norvig Corrected: i have just received the letter which lets me know that i have won the first prize (Acc: 1.00)

Clean: yours sincerely
Noisy: yours sincerly
My Corrected 1: yours sincerely (Acc: 1.00)
My Corrected 2: yours sincerely (Acc: 1.00)
My Corrected (No Keyboard): yours sincerely (Acc: 1.00)
My Corrected 3: yours sincerely (Acc: 1.0

## Visualization of Results

Below is a bar plot comparing the average accuracies of the spelling correction methods on the BEA-4K dataset:

<img src="accuracy_comparison.png" alt="Average Accuracy Comparison" width="1400" >

### Interpretation
- `Correction 1` achieves the highest accuracy (0.964859), followed closely by `Correction 2` and `Correction 2 (No Keyboard)` (both 0.962504).
- Norvig’s method has the lowest accuracy (0.960724), highlighting the benefit of context-aware corrections.

## Analysis of Second Test Set Results (BEA-4K Dataset)

### Benchmark Results
The average accuracies for the correction algorithms, evaluated on the BEA-4K dataset (`test.bea4k` clean and `test.bea4k.noise`), are as follows:

- **Average My Accuracy 1:** 0.964859
- **Average My Accuracy 2:** 0.962504
- **Average My Accuracy 2 (No Keyboard):** 0.962504
- **Average My Accuracy 3:** 0.962402
- **Average Norvig Accuracy:** 0.960724

### Sample Outputs
Below are three example sentence pairs from the evaluation, showing the clean text, noisy input, and the outputs of each correction method along with their per-sentence accuracies:

#### Example 1
- **Clean:** `i have given a questionnaire to other students in my class to know their preferences regarding this choice and we all believe that the first lesson that should be filmed is philosophy`
- **Noisy:** `i have given a questionnaire to other students in my class to know their preferences regarding this choise and we all believe that the first lesson that should be filmed is philosophy`
- **My Corrected 1:** `i have given a questionnaire to other students in my class to know their preference regarding this choice and we all believe that the first lesson that should be filled is philosophy` (Acc: 0.94)
- **My Corrected 2:** `i have given a questionnaire to other students in my class to know their preference regarding this choice and we all believe that the first lesson that should be filled is philosophy` (Acc: 0.94)
- **My Corrected (No Keyboard):** `i have given a questionnaire to other students in my class to know their preference regarding this choice and we all believe that the first lesson that should be filled is philosophy` (Acc: 0.94)
- **My Corrected 3:** `i have given a questionnaire to other students in my class to know their preference regarding this choice and we all believe that the first lesson that should be filled is philosophy` (Acc: 0.94)
- **Norvig Corrected:** `i have given a questionnaire to other students in my class to know their references regarding this choose and we all believe that the first lesson that should be filled is philosophy` (Acc: 0.91)

#### Example 2
- **Clean:** `first i think we should rent bigger halls so that we can make a better sound and give more space for the audience`
- **Noisy:** `first i think we shoul rent bigger halls so that we can make a better sound and give more space for the audience`
- **My Corrected 1:** `first i think we should rent bigger halls so that we can make a better sound and give more space for the audience` (Acc: 1.00)
- **My Corrected 2:** `first i think we shout rent bigger halls so that we can make a better sound and give more space for the audience` (Acc: 0.96)
- **My Corrected (No Keyboard):** `first i think we shout rent bigger halls so that we can make a better sound and give more space for the audience` (Acc: 0.96)
- **My Corrected 3:** `first i think we shout rent bigger halls so that we can make a better sound and give more space for the audience` (Acc: 0.96)
- **Norvig Corrected:** `first i think we should rent bigger halls so that we can make a better sound and give more space for the audience` (Acc: 1.00)

#### Example 3
- **Clean:** `it s a great opportunity for me to participate in your camp california because normally i work a lot and i ca n t spend money on travel moreover i have to support a big family because i m married and i have three children`
- **Noisy:** `it s a great opportunity for me to participate in your camp california because normaly i work a lot and i ca n t spend money on travel moreover i have to support a big family because i m married and i have three children`
- **My Corrected 1:** `it s a great opportunity for me to participate in your camp california because normally i work a lot and i ca n t spend money on travel moreover i have to support a big family because i m married and i have three children` (Acc: 1.00)
- **My Corrected 2:** `it s a great opportunity for me to participate in your camp california because normally i work a lot and i ca n t spend money on travel moreover i have to support a big family because i m married and i have three children` (Acc: 1.00)
- **My Corrected (No Keyboard):** `it s a great opportunity for me to participate in your camp california because normally i work a lot and i ca n t spend money on travel moreover i have to support a big family because i m married and i have three children` (Acc: 1.00)
- **My Corrected 3:** `it s a great opportunity for me to participate in your camp california because normally i work a lot and i ca n t spend money on travel moreover i have to support a big family because i m married and i have three children` (Acc: 1.00)
- **Norvig Corrected:** `it s a great opportunity for me to participate in your camp california because normal i work a lot and i ca n t spend money on travel moreover i have to support a big family because i m married and i have three children` (Acc: 0.98)

### Analysis

#### Overall Performance
- **Correction 1 Leads:** `Correction 1` achieves the highest average accuracy (0.964859), outperforming all other methods, including Norvig’s solution (0.960724). This suggests that its combination of bigram context and keyboard-aware edits is particularly effective for the human-like errors in the BEA-4K dataset.
- **Close Competition:** `Correction 2`, `Correction 2 (No Keyboard)`, and `Correction 3` have similar accuracies (0.962504, 0.962504, and 0.962402, respectively), slightly below `Correction 1` but still competitive with Norvig’s 0.960724. This indicates that higher N-gram models (used in `Correction 2` and `3`) provide marginal benefits over Norvig’s unigram approach but may not fully leverage the dataset’s error patterns.
- **Norvig’s Baseline:** Norvig’s accuracy (0.960724) is the lowest, though still strong, reflecting its reliance on unigram probabilities without context or keyboard-awareness, which limits its ability to handle certain errors effectively.

#### Sample-Specific Insights
1. **Example 1 (choise → choice):**
   - All my methods correctly fix "choise" to "choice" and adjust "preferences" to "preference" but incorrectly change "filmed" to "filled" (likely due to N-gram weighting favoring "filled" in some contexts). Accuracy is 0.94 for all my methods.
   - Norvig fails on "choise" (outputs "choose") and "preferences" (outputs "references"), resulting in a lower accuracy of 0.91. This highlights the advantage of context-awareness in my methods.

2. **Example 2 (shoul → should):**
   - `Correction 1` and Norvig perfectly correct "shoul" to "should" (Acc: 1.00), showing their robustness for simple typos.
   - `Correction 2`, `Correction 2 (No Keyboard)`, and `Correction 3` incorrectly output "shout" (Acc: 0.96), possibly due to over-reliance on higher N-grams or insufficient keyboard context to disambiguate "shoul". This suggests a potential weighting issue in these methods for this specific case.

3. **Example 3 (normaly → normally):**
   - All my methods perfectly correct "normaly" to "normally" (Acc: 1.00), demonstrating consistent performance on straightforward insertion errors.
   - Norvig outputs "normal" (Acc: 0.98), missing the full correction, likely because "normal" has a higher unigram probability than "normally" without context to guide it.

#### Keyboard vs. No Keyboard
- Surprisingly, `Correction 2` and `Correction 2 (No Keyboard)` have identical average accuracies (0.962504), despite BEA-4K containing human-like typos. This could indicate:
  - The keyboard-adjacent edit feature isn’t being fully utilized, possibly due to implementation details (e.g., limited keyboard edit candidates or insufficient weighting).
  - The dataset’s errors may not predominantly involve keyboard adjacency (e.g., more insertions like "normaly" than swaps like "teh"), reducing the feature’s impact.
- Sample outputs (e.g., "shoul" → "should" vs. "shout") suggest some differentiation, but the overall effect is neutralized across the dataset.

#### Strengths and Weaknesses
- **Strengths:**
  - `Correction 1` excels due to its balanced use of bigrams and keyboard edits, making it the most robust for this dataset.
  - All my methods outperform Norvig on average, validating the value of context (N-grams) over unigrams alone.
- **Weaknesses:**
  - Higher N-gram methods (`Correction 2` and `3`) occasionally overcorrect (e.g., "should" → "shout"), suggesting that trigram/fourgram data might be sparse or misweighted in some cases.
  - The lack of differentiation between keyboard and no-keyboard methods indicates a need to refine the keyboard edit feature or test on a dataset with more adjacency-specific errors.



## Conclusion

<p style="font-size: 24px;">
BEA-4K test confirms that my algorithms, particularly `Correction 1`, improve upon Norvig’s solution by leveraging context and keyboard-awareness, achieving a peak accuracy of 0.964859 versus Norvig’s 0.960724. However, the identical performance of `Correction 2` and its no-keyboard variant suggests that the keyboard feature’s potential isn’t fully realized, possibly due to the nature of the errors in BEA-4K or implementation limitations. Future work could involve tuning N-gram weights, enhancing keyboard edit candidate generation, or testing on a dataset with more pronounced keyboard-specific typos to further distinguish these methods.
</p> 

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)